# Neural Networks for Ligand-based Screening

*adapted from Volkamer Lab, 2020*

## The Problem

The epidermal growth factor receptor (EGFR) is a transmembrane protein involved in cell development and in innate immune response in human skin. EGFR requires specific ligands to become activated. Overexpression of EFGR has been linked to numerous types of cancer, where constant activation of EFGRs lead to uncontrolled cell division. 

Inhibitors of EGFRs have become the target of anticancer therapeutic development. If a drug can bind to overexpressed EFGRs, then the required signal molecules cannot bind EGFR and the signaling cascade resulting in growth/proliferation is stopped. While several drugs, including osimertinib, gefitinib, erlotinib, and bregatinib, have been developed to target EGFRs, patients develop resistance, however. Development of new drugs to target EGFR and its mutations is an active area of drug development.

## Our Assignment: *in silico* ligand screening

In this notebook, we will build a neural network model to perform ligand-based screening of EGFR inhibitors. There are typically several ways to do this. One approach is to perform molecular dynamics and docking simluations, to computationally determine how well a particular drug candidate binds to a target. This approach is useful, but typically too computationally demanding to apply to thousands of drug candidates. A better first tool is to use a ML-based approach, which can quickly screen through thousands of candidate drugs to produce a small set of primary leads. 

When building such a model, we have to decide exactly what to predict. A common property to use in drug discovery is the **IC50**. The IC50, or the half-maximal inhibitory concentration, is the concentration of a drug needed to reduce a target biological activity by half. Because these values can span a very wide range, the most common choice in ML models is the **pIC50**, which is the negative log of the IC50. The main difficulty of using pIC50s in ML models is data availability.

## Building Neural Networks

To train a NN model, we follow essentially the same steps that we did for other supervised learning models. The main difference is that we now have more hyperparameters to deal with, and we need to more carefully tune how well the model is performing.

We will need to use some software not native on ChemCompute. There are two main choices here, and we will use TensorFlow (PyTorch is the other option). Execute the cells below to perform the installation. It may take a few minutes.

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys, Draw, rdFingerprintGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

## Loading the Data
We will use experimentally determined IC50s, converted to pIC50s, in our model. These are stored in the file `CHEMBL25_activities_EGFR.csv` in the data folder. In the cell below, load this data into a dataframe. 

Now, generate the MACCS fingerprints for all molecules in the dataset. Add these fingerprints as a new column in the dataframe. Print the first few lines of the dataframe to ensure it is correct.

Let's only use these fingerprints as our descriptors, so there's no need for feature selection or normalization. All we need to do next is get our training and testing subsets, do that in the cell below.

## Define and Compile the Model

Before actually training the model, we need to define and compile it. This step was a simple one-liner for our regression models, but here we'll need to get into the details a little bit more.

Let's define and complile a model with two hidden layers with 64 and 32 nodes, respectively.

In [ ]:
#initialize
model = Sequential()

# First hidden layer
model.add(Dense(64, activation="relu", name="layer1"))
# Second hidden layer
model.add(Dense(32, activation="relu", name="layer2"))
# Output layer
model.add(Dense(1, activation="linear", name="layer3"))


Note that the output layer has a size of one (for our single-value output), and that the activation fuction goes to linear. Finally, in the cell below, we'll compile the model. This is where we define our loss function and our optimizer. 

In [ ]:
# Compile
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse", "mae"])

## Training the model
Now we need to train the model. Along with specifying the training features and labels, we need to set a few additional parameters:

 - Batch Size. Defines the subset of data to use to calculate the gradient in optimizing the loss function.
 - Epochs. This is the number of times the model will traverse the network. You need a sufficient number of epochs to effectively minimize the loss function, but too many become computationally demanding.

Unlike our previous work, we'll need to analyze the performance of the fit. This entails analyzing validation errors as a function of epoch. Complete the function below by filling in the missing information.

In [ ]:
history = model.fit(
    , # x values
    , # y values
    batch_size=, # batch size
    validation_data=, # validation data
    verbose=0,
    epochs=, # number of epochs
)

Excecute the cell below to analyze the optimization. You should see a plot of training and validation errors.

In [ ]:
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="test")
plt.legend(["train", "test"], loc="upper right")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.ylim((0, 15))
plt.title(
    f"test loss = {history.history['val_loss'][nb_epoch-1]:.2f}, " f"batch size = {batch}"
)

Using the above cells, try to improvy your testing error by testing a few different batch sizes. 

We'll want to save the weights from the best performing epoch. This is automated by creating a `ModelCheckpoint`. In the cell below, I've added some code to save the best weights to a file. Complete the fitting function with you data and an optimal batch size you determined earlier, then execute the cell.

In [ ]:
# Save the trained model
filepath = "best_weights.hdf5"
checkpoint = ModelCheckpoint(
    str(filepath),
    monitor="loss",
    verbose=0,
    save_best_only=True,
    mode="min",
    save_weights_only=True,
)
callbacks_list = [checkpoint]

# Fit the model
model.fit(
    , # x values
    , # y values
    epochs=,
    batch_size=,
    callbacks=callbacks_list,
    verbose=0,
)

Once you have your final model trained, we can save it to a file, so that we don't need to re-train anything to use it. Excecute the cell beow to save your model. Feel free to give the file a more descriptive name.

In [ ]:
model.save("my_model.keras")

## Evaluating the Model

The plots we made above are very useful to analyze our model's performance, but there's more we can do, like calculating other evaluation metrics and visualizing our testing performance.

In the cell below, generate a list of predicted pIC50s for your test set. You will want to `flatten` the result, so that the array is the same shape as your testing labels.

Calculate the mean absolute error and the root mean squared error for these predictions. 

Make a scatter plot of these predictions using matplotlib. Have your predicted values on the $x$-axis, and your true values on the $y$-axis. Also plot the line $y=x$ with the data.

Use your data to calulate an $R^2$ value for the correlation between your predicted values and the true values.